In [1]:
import numpy as np, tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os, numpy as np, pandas as pd

KeyboardInterrupt: 

In [3]:
folder = "tep"
files = sorted([f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))])

X_list = []
for f in files:
    temp = pd.read_csv(f"{folder}/{f}").to_numpy()
    temp = temp[:, 1:] # ensure matrix, no header row
    X_list.append(temp)

X = np.stack(X_list, axis=0)
X = X[..., None]
print(X.shape)

(800, 300, 50, 1)


In [4]:
inp = keras.Input(shape=(X.shape[1], X.shape[2], 1), name="encoder_input")

# Encoder (all CNN)
x = layers.Conv2D(32, 3, strides=2, padding="same", activation="relu")(inp)
x = layers.Conv2D(64, 3, strides=2, padding="same", activation="relu")(x)
x = layers.Conv2D(128,3, strides=2, padding="same", activation="relu")(x)
latent_map = layers.Conv2D(256,3, strides=2, padding="same", activation="relu", name="latent_map")(x)

# Decoder
y = layers.Conv2DTranspose(128,3, strides=2, padding="same", activation="relu")(latent_map)
y = layers.Conv2DTranspose(64, 3, strides=2, padding="same", activation="relu")(y)
y = layers.Conv2DTranspose(32, 3, strides=2, padding="same", activation="relu")(y)
y = layers.Conv2DTranspose(1,  3, strides=2, padding="same")(y)
y = layers.Activation("sigmoid", name="recon_prob")(y)
y = layers.CenterCrop(X.shape[1], X.shape[2])(y)

auto = keras.Model(inp, y, name="autoencoder")


auto.compile(optimizer=keras.optimizers.Adam(1e-3),
             loss=keras.losses.BinaryCrossentropy(from_logits=False),
             metrics=[keras.metrics.BinaryAccuracy(threshold=0.5,name="bin_acc")])

auto.summary()

history = auto.fit(
    X, X,
    epochs=30,
    batch_size=min(64, X.shape[0]),
    validation_split=0.2,
    verbose=1
)


enc_out = auto.get_layer("latent_map").output
encoder_flat = keras.Model(
    inputs=auto.input,
    outputs=layers.Flatten(name="latent_flat")(enc_out),
    name="encoder_flat",
)

# Z = encoder_flat.predict(X, verbose=0)


Model: "autoencoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ encoder_input (InputLayer)      │ (None, 300, 50, 1)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 150, 25, 32)    │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 75, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 38, 7, 128)     │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ latent_map (Conv2D)             │ (None, 19, 4, 256)     │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose                │ (None, 38, 8, 128)     │       295,040 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_1              │ (None, 76, 16, 64)     │        73,792 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_2              │ (None, 152, 32, 32)    │        18,464 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_3              │ (None, 304, 64, 1)     │           289 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ recon_prob (Activation)         │ (None, 304, 64, 1)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ center_crop (CenterCrop)        │ (None, 300, 50, 1)     │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 775,425 (2.96 MB)

 Trainable params: 775,425 (2.96 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - bin_acc: 0.7883 - loss: 0.6875 - val_bin_acc: 0.8328 - val_loss: 0.5827
Epoch 2/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - bin_acc: 0.8987 - loss: 0.4986 - val_bin_acc: 0.8410 - val_loss: 0.4794
Epoch 3/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - bin_acc: 0.9050 - loss: 0.3511 - val_bin_acc: 0.8410 - val_loss: 0.4091
Epoch 4/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - bin_acc: 0.9047 - loss: 0.2864 - val_bin_acc: 0.8410 - val_loss: 0.3692
Epoch 5/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 16s 2s/step - bin_acc: 0.9051 - loss: 0.2605 - val_bin_acc: 0.8414 - val_loss: 0.3587
Epoch 6/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - bin_acc: 0.9077 - loss: 0.2479 - val_bin_acc: 0.8489 - val_loss: 0.3434
Epoch 7/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 16s 2s/step - bin_acc: 0.9113 - loss: 0.2367 - val_bin_acc: 0.8605 - val_loss: 0.3124
Epoch 8/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 16s 2s/step - bin_acc: 0.9192 - loss: 0.2227 - val_bin_acc: 0.8833 - val_loss: 0.2804
Epoch 9/

In [5]:
def latent_l2_distance(A: np.ndarray, B: np.ndarray, encoder: keras.Model) -> float:
    """
    Compute L2 distance between latent representations of two 2D binary arrays.
    A, B shape: (m, v). Encoder is from build_and_train_cnn_autoencoder.
    """
    assert A.ndim == 2 and B.ndim == 2, "A and B must be 2D arrays"
    A_in = A.astype(np.float32)[None, ..., None]  # (1, m, v, 1)
    B_in = B.astype(np.float32)[None, ..., None]  # (1, m, v, 1)
    zA = encoder.predict(A_in, verbose=0)[0]
    zB = encoder.predict(B_in, verbose=0)[0]
    return float(np.linalg.norm(zA - zB, ord=2))

In [37]:
latent_l2_distance(X[10, :, :, 0], X[550, :, :, 0], encoder_flat)

245.53582763671875